In [ ]:
!pip install pysentimiento
!pip install emoji
!pip install -U pycld3

In [3]:
#@title Set up Directory { run: "auto"}
import os
from IPython.display import clear_output
from google.colab import drive 
from IPython.display import clear_output
drive.mount('/content/gdrive', force_remount = True)
working_directory = 'My Drive/COVID19_Tweets' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)

Mounted at /content/gdrive
current directory is : /content/gdrive/My Drive/COVID19_Tweets


# Dataset não rotulado

In [ ]:
# import cld3
# import pandas as pd

# def lang_detector(x):
#   try:
#     return cld3.get_language(x)
#   except:
#     return '?'

# df = pd.read_csv('/content/mexico_global.csv')
# df.dropna(inplace = True)
# df.drop(['Unnamed: 0'], axis = 1, inplace = True)
# df['lang'] = df['body'].apply(lambda x: lang_detector(x))
# df['lang'] = df['lang'].apply(lambda x: x[0])
# df = df.drop(df[(df['author'] == 'AutoModerator') | (df['author'] == '[deleted]')].index)
# df.drop_duplicates(subset='body', keep='first', inplace=True)

# df.head()

In [ ]:
# from pysentimiento import create_analyzer
# from pysentimiento.preprocessing import preprocess_tweet

# analyzer = create_analyzer(task="sentiment", lang="en")
# df['body'] = df['body'].apply(lambda x: preprocess_tweet(x))
# df['pred'] = df['body'].apply(lambda x: analyzer.predict(x))

In [ ]:
# df['label'] = df['pred'].apply(lambda x: x.output)
# df['probas'] = df['pred'].apply(lambda x: x.probas)

In [ ]:
# df['label'].value_counts()

NEG    4427
NEU    3606
POS     624
Name: label, dtype: int64

# Dataset rotulado (baseline)

In [13]:
df_rotulado = pd.read_csv('/content/gdrive/MyDrive/COVID19_Tweets/amostra_rotulada.csv').drop(['Unnamed: 0'], axis = 1)
df_rotulado = df_rotulado.sample(5000)

In [14]:
from pysentimiento import create_analyzer
from pysentimiento.preprocessing import preprocess_tweet

analyzer = create_analyzer(task="sentiment", lang="en")
df_rotulado['text'] = df_rotulado['text'].apply(lambda x: preprocess_tweet(x))
df_rotulado['pred'] = df_rotulado['text'].apply(lambda x: analyzer.predict(x))
df_rotulado['label'] = df_rotulado['pred'].apply(lambda x: x.output)
df_rotulado['probas'] = df_rotulado['pred'].apply(lambda x: x.probas)

In [17]:
df_rotulado['Sentiment_Label'] = df_rotulado['Sentiment_Label'].map({ 'neutral': 'NEU',
                                                                      'negative': 'NEG',
                                                                      'positive': 'POS'})
df_rotulado.head()

,Tweet_ID,Sentiment_Label,text,pred,label,probas
94694,1220399453158293505,NEU,RT @usuario: China allocates 1 billion yuan ($...,"AnalyzerOutput(output=NEU, probas={NEU: 0.938,...",NEU,"{'NEG': 0.0020287414081394672, 'NEU': 0.937936..."
128971,1220457095620976642,NEG,RT @usuario: 🤦 emoji tono de piel claro medio ...,"AnalyzerOutput(output=NEG, probas={NEG: 0.963,...",NEG,"{'NEG': 0.9630939364433289, 'NEU': 0.034001730..."
61230,1220347645606055936,NEU,RML Hospital In Delhi Sets Up First Corona Vir...,"AnalyzerOutput(output=NEU, probas={NEU: 0.978,...",NEU,"{'NEG': 0.0028543630614876747, 'NEU': 0.978310..."
67761,1220362114067464193,NEU,RT @usuario: China appears to be locking down ...,"AnalyzerOutput(output=NEU, probas={NEU: 0.949,...",NEU,"{'NEG': 0.042912572622299194, 'NEU': 0.9485344..."
7037,1220013579275055108,NEU,RT @usuario: WHO interim guidance on a range o...,"AnalyzerOutput(output=NEU, probas={NEU: 0.829,...",NEU,"{'NEG': 0.0020122884307056665, 'NEU': 0.828626..."


In [33]:
from sklearn.metrics import f1_score, recall_score, precision_score

y_true = df_rotulado['Sentiment_Label']
y_pred = df_rotulado['label']

print(f'F1 score: {round(f1_score(y_true, y_pred, average = "macro"), 4)}')
print(f'Recall: {round(recall_score(y_true, y_pred, average = "weighted"), 4)}')
print(f'Precision: {round(precision_score(y_true, y_pred, average = "weighted"), 4)}')

F1 score: 0.6069
Recall: 0.6956
Precision: 0.7124
